In [1]:
! pip install pdfservices-sdk -U

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import re

import glob
import json
import shutil
from tqdm.notebook import tqdm
from zipfile import ZipFile


from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import ExtractRenditionsElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.table_structure_type import TableStructureType
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation


In [4]:
# Initial setup, create credentials instance.
credentials = Credentials.service_principal_credentials_builder(). \
    with_client_id(os.getenv('PDF_SERVICES_CLIENT_ID')). \
    with_client_secret(os.getenv('PDF_SERVICES_CLIENT_SECRET')). \
    build()

# Create an ExecutionContext using credentials and create a new operation instance.
execution_context = ExecutionContext.create(credentials)
extract_pdf_operation = ExtractPDFOperation.create_new()


In [5]:
# Filter protocol only pdf
datadir = os.getcwd()
filenames = glob.glob(os.path.join(datadir, r"*.pdf"))
# pattern = r'NCT\d+_Prot_\d+.pdf'
# filenames = [filename for filename in filenames if re.match(pattern, os.path.basename(filename))]
display(len(filenames))

1

In [6]:
for filename in tqdm(filenames):
    # Set operation input from a source file.
    # filename = "NCT01687673_Prot_SAP.pdf"
    # filepath = os.getcwd()

    source = FileRef.create_from_local_file(filename)
    extract_pdf_operation.set_input(source)

    # Build ExtractPDF options and set them into the operation
    # extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
    #     .with_elements_to_extract([ExtractElementType.TEXT, ExtractElementType.TABLES]) \
    #     .with_element_to_extract_renditions(ExtractRenditionsElementType.TABLES) \
    #     .with_table_structure_format(TableStructureType.CSV) \
    #     .build()
    extract_pdf_options: ExtractPDFOptions = ExtractPDFOptions.builder() \
        .with_element_to_extract(ExtractElementType.TEXT) \
        .build()
    extract_pdf_operation.set_options(extract_pdf_options)

    # Execute the operation.
    result: FileRef = extract_pdf_operation.execute(execution_context)

    # Save the result to the specified location.
    result.save_as(filename.replace(".pdf", ".zip"))

    with ZipFile(filename.replace(".pdf", ".zip"), 'r') as f:
        f.extractall(os.getcwd() )
        
    # Move structuredData.json to pdf/filename.json
    shutil.move('structuredData.json', filename.replace(".pdf", ".json"))